In [1]:
import requests
import numpy as np
from lxml import etree
from datetime import  date,datetime, timedelta
import xml.etree.ElementTree as ET
from operator import itemgetter

In [2]:
torneos = ["Champions","Copa America", "Copa Paraguay", "Eliminatorias","España","Italia","Libertadores","Paraguay","Premier League","Recopa","Sudamericana"]
torneovec =["champions","copaamerica", "copaparaguay", "eliminatorias","espana","italia","libertadores","paraguay","premierleague","recopa","sudamericana"]
estadisticas = ["posiciones","fixture","jugadores.estadisticas"]
estadistica_top = {
    "Goleadores" : "Goals",
    "Asistencias": "GoalAssistances",
    "Recuperacion" : "Stealing",
    "Faltas Cometidas" : 'Fouls',
    "Faltas Recibidas" : 'FoulsOpponents',
    "Tarjetas Amarillas" : 'YellowCards',
    "Tarjetas Rojas" : 'RedCards'
}
x='-1111'
while int(x) < 0  or int(x) > 10:
    print("Seleccionar torneo:")
    for i in range(len(torneos)):
        print(str(i) + "- " + torneos[i])
    x = input()
torneo = torneovec[int(x)]

Seleccionar torneo:
0- Champions
1- Copa America
2- Copa Paraguay
3- Eliminatorias
4- España
5- Italia
6- Libertadores
7- Paraguay
8- Premier League
9- Recopa
10- Sudamericana
8


In [3]:
#funciones
def traer_datos(torneo,estadistica):
    url = f'https://editorialdenegocios.com/datafactory/xml/es/{torneo}/deportes.futbol.{torneo}.{estadistica}.xml'
    print(url)
    web = requests.get(url)
    root = ET.fromstring(web.text)
    return root
def posiciones():
    if torneo != 'sudamericana' and torneo != 'libertadores':
        root = traer_datos(torneo,estadisticas[0])
        if torneo == 'champions' :#ordenar tabla solo para champions y libertadores
            root[:] = sorted(root, key=lambda child: (child.tag,child.get('zona')))
            for c in root:
                c[:] = sorted(c, key=lambda child: (child.tag,child.get('puntos')))
        
        f.write('Tabla de posiciones ; PJ ; G ; E ; P ; GF ; GC ; DF ; Pts;\n')
        contador=0
        for i in root.findall('equipo'):
            if contador % 4 == 0 and (torneo == 'champions' or torneo == 'libertadores' or torneo == 'sudamericana'):
                f.write(i.get("zona") + "\n")
    
            f.write(i.find('nombre').text + ';' + i.find('jugados').text + ';' + i.find('ganados').text + ';' + i.find('empatados').text + ';' + i.find('perdidos').text +  ';' + i.find('golesfavor').text + ';' + i.find('golescontra').text + ';' + i.find('difgol').text + ';' + i.find('puntos').text +'\n')
            contador = contador + 1
def fixture():
    root = traer_datos(torneo,estadisticas[1])
    for i in root.findall("fecha"):
        if(i.get('estado')=='ultima'):
            f.write(i.get('nombrenivel')+";-;"+i.get('nombre')+"\n")
            for j in i:
                if(j.find('local')!= None):
                    if j.find('estado').text == 'Postergado':
                        f.write(f'{j.find("local").text};;{j.find("estado").text};;{j.find("visitante").text};{j.get("tipo")}\n')
                    else:
                        f.write(f'{j.find("local").text};{j.find("goleslocal").text}; - ;{j.find("golesvisitante").text} ;{j.find("visitante").text};{j.get("tipo")}\n') 
        if(i.get('estado')=='actual'):
            f.write(i.get('nombrenivel')+";-;"+i.get('nombre')+"\n")  
            for j in i:
                if(j.find('local')!= None):
                    if (j.find('estado').attrib['id'] == '2'):
                        f.write(f'{j.find("local").text};{j.find("goleslocal").text}; - ;{j.find("golesvisitante").text} ;{j.find("visitante").text} - {j.get("tipo")}; - ;fecha:{j.get("fecha")[6:8]}/{j.get("fecha")[4:6]}/{j.get("fecha")[0:4]}; hora:{j.get("hora")}\n')
                    else:
                        f.write(f'{j.find("local").text};; vs ;;{j.find("visitante").text} - {j.get("tipo")}; - ;fecha:{j.get("fecha")[6:8]}/{j.get("fecha")[4:6]}/{j.get("fecha")[0:4]}; hora:{j.get("hora")}\n')
       
        if(i.get('estado')=='proxima'):
            f.write(i.get('nombrenivel')+";-;"+i.get('nombre')+"\n")  
            for j in i:
                if(j.find('local')!= None):
                    if (j.find('estado').attrib['id'] == '2'):
                        f.write(f'{j.find("local").text};{j.find("goleslocal").text}; - ;{j.find("golesvisitante").text} ;{j.find("visitante").text} - {j.get("tipo")}; - ;fecha:{j.get("fecha")[6:8]}/{j.get("fecha")[4:6]}/{j.get("fecha")[0:4]}; hora:{j.get("hora")}\n')
                    else:
                        f.write(f'{j.find("local").text};; vs ;;{j.find("visitante").text} - {j.get("tipo")}; - ;fecha:{j.get("fecha")[6:8]}/{j.get("fecha")[4:6]}/{j.get("fecha")[0:4]}; hora:{j.get("hora")}\n')
       


def cargar_vector_top(estadistica):
    if estadistica != "Pases":
        vector_top = np.array([0,-1,0],dtype=object)
    else:
        vector_top = np.array([0,-1,0,0, 0],dtype=object)
    for i in root.findall('Tournament/Players/Player'):
        if estadistica != "Pases":
            text = [i.attrib['name'] + " " + i.attrib['lastName'], int(i.find(estadistica_top[estadistica]).attrib['total']) , i.attrib['currentTeam']]
        else:
            text = [i.attrib['name'] + " " + i.attrib['lastName'], int(i.find('Passes').attrib['successful']) , int(i.find('Passes').attrib['total']), float(i.find('Passes').attrib['effectiveness'])  , i.attrib['currentTeam']]
        vector_top = np.vstack([vector_top,np.asarray(text,object)]) 
    f.write(f'\n {estadistica} \n')
    if estadistica != 'Pases':
        vector_top = sorted(vector_top, key=lambda a_entry: a_entry[1], reverse=True)
    else: 
        vector_top = sorted(vector_top, key=lambda a_entry: a_entry[2], reverse=True)
        pases10 = np.array([0,-1,0,0, 0],dtype=object)
        for i in range(11):
            pases10 = np.vstack([pases10,vector_top[i]])  
            pases10[i][3] = int(pases10[i][3])
    
        pases10 = sorted(pases10, key=lambda a_entry: a_entry[3], reverse=True)
        vector_top = pases10

        
    for i in range(10):
        a=vector_top[i]
        if estadistica != "Pases":
            f.write(a[0]+ ";" +str(a[1]) + ";" + a[2] +"\n")
        else:
            f.write(a[0]+ ";" +str(a[1]) + ";" + str(a[2]) + ";" + str(a[3]) + ";" + a[4] +"\n")
       

In [4]:
titulocsv = f'datos{torneo}.csv'
f = open(titulocsv,'w')
posiciones()
fixture()
root = traer_datos(torneo,estadisticas[2])
cargar_vector_top("Goleadores")
cargar_vector_top("Asistencias")
cargar_vector_top("Pases")
cargar_vector_top("Recuperacion")
cargar_vector_top("Faltas Cometidas")
cargar_vector_top("Faltas Recibidas")
cargar_vector_top("Tarjetas Amarillas")
cargar_vector_top("Tarjetas Rojas")
f.close()

https://editorialdenegocios.com/datafactory/xml/es/premierleague/deportes.futbol.premierleague.posiciones.xml
https://editorialdenegocios.com/datafactory/xml/es/premierleague/deportes.futbol.premierleague.fixture.xml
https://editorialdenegocios.com/datafactory/xml/es/premierleague/deportes.futbol.premierleague.jugadores.estadisticas.xml


In [5]:
estadisticas_dict

NameError: name 'estadisticas_dict' is not defined

In [5]:
print('Asociación Nacional RepublicanaTribunal Electoral Partidario\nArea de SistemasElección Interna Partidaria del \n20/06/2021 - Municipa\nles del 10/10/2021\nAsunción, 5 de marzo de 2021Anexo de la Resolución N° 183 / 2021 Mov.:\nFRENTE UNIVERSITARIO DE ALTO PARANA\nMFUDAPSiglas:Depto.:10ALTO PARANA\nDistrito:0CIUDAD DEL ESTE\n Cargo:CONCEJAL SUPLENTE\n3OrdenC.I. NºNombre y Apellido\n13.371.132MARIA MAGDALENA RIVEROS LEON\n25.240.438CRISTIAN LUIS FARIÑA MARTINEZ\n34.154.431MARIA VILMA VILLAGRA RODAS\n42.442.763FREDDY ENRIQUE ZARACHO DUARTE\n54.379.520MOISES ZACARIAS MARIN ACOSTA\n6724.881ALCIDES GUSTAVO CHILAVERT GONZALEZ\n74.728.086SUSANA ELIZABETH MENDOZA BELLOSO\n83.336.137RUBEN GARCIA AGUILAR\n93.661.207BLAS ARIEL FRANCO LEZCANO\n105.040.135KARINA CONCEPCION LOPEZ VAZQUEZ\n115.140.311\nJULIA YANINA ORTIZ GONZALEZ\n124.164.134EDGAR MIGUEL RODRIGUEZ VAZQUEZ\n')

Asociación Nacional RepublicanaTribunal Electoral Partidario
Area de SistemasElección Interna Partidaria del 
20/06/2021 - Municipa
les del 10/10/2021
Asunción, 5 de marzo de 2021Anexo de la Resolución N° 183 / 2021 Mov.:
FRENTE UNIVERSITARIO DE ALTO PARANA
MFUDAPSiglas:Depto.:10ALTO PARANA
Distrito:0CIUDAD DEL ESTE
 Cargo:CONCEJAL SUPLENTE
3OrdenC.I. NºNombre y Apellido
13.371.132MARIA MAGDALENA RIVEROS LEON
25.240.438CRISTIAN LUIS FARIÑA MARTINEZ
34.154.431MARIA VILMA VILLAGRA RODAS
42.442.763FREDDY ENRIQUE ZARACHO DUARTE
54.379.520MOISES ZACARIAS MARIN ACOSTA
6724.881ALCIDES GUSTAVO CHILAVERT GONZALEZ
74.728.086SUSANA ELIZABETH MENDOZA BELLOSO
83.336.137RUBEN GARCIA AGUILAR
93.661.207BLAS ARIEL FRANCO LEZCANO
105.040.135KARINA CONCEPCION LOPEZ VAZQUEZ
115.140.311
JULIA YANINA ORTIZ GONZALEZ
124.164.134EDGAR MIGUEL RODRIGUEZ VAZQUEZ



In [6]:
import re

In [42]:
text='13.371.132MARIA MAGDALENA RIVEROS LEON\n25.240.438CRISTIAN LUIS FARIÑA MARTINEZ\n34.154.431MARIA VILMA VILLAGRA RODAS\n42.442.763FREDDY ENRIQUE ZARACHO DUARTE\n54.379.520MOISES ZACARIAS MARIN ACOSTA\n6724.881ALCIDES GUSTAVO CHILAVERT GONZALEZ\n74.728.086SUSANA ELIZABETH MENDOZA BELLOSO\n83.336.137RUBEN GARCIA AGUILAR\n93.661.207BLAS ARIEL FRANCO LEZCANO\n105.040.135KARINA CONCEPCION LOPEZ VAZQUEZ\n115.140.311\nJULIA YANINA ORTIZ GONZALEZ\n124.164.134EDGAR MIGUEL RODRIGUEZ VAZQUEZ\n'

In [94]:
import re
text='13.371.132MARIA MAGDALENA RIVEROS LEON\n25.240.438CRISTIAN LUIS FARIÑA MARTINEZ\n34.154.431MARIA VILMA VILLAGRA RODAS\n42.442.763FREDDY ENRIQUE ZARACHO DUARTE\n54.379.520MOISES ZACARIAS MARIN ACOSTA\n6724.881ALCIDES GUSTAVO CHILAVERT GONZALEZ\n74.728.086SUSANA ELIZABETH MENDOZA BELLOSO\n83.336.137RUBEN GARCIA AGUILAR\n93.661.207BLAS ARIEL FRANCO LEZCANO\n105.040.135KARINA CONCEPCION LOPEZ VAZQUEZ\n115.140.311\nJULIA YANINA ORTIZ GONZALEZ\n124.164.134EDGAR MIGUEL RODRIGUEZ VAZQUEZ\n'
filtro_nombre=re.split('[\d]',text)
filtro_cedula=re.split('[^\d.]',text)
nombres=[]
cedula=[]
for (index,item) in enumerate(filtro_nombre):
    item=item.replace('\n','')
    if(item!='' and item!='.'):
        nombres.append(item)
for (index,item) in enumerate(filtro_cedula):
    item=item.replace('\n','')
    if(item!='' and item!='.'):
        cedula.append(item)


In [92]:
nombres

['MARIA MAGDALENA RIVEROS LEON',
 'CRISTIAN LUIS FARIÑA MARTINEZ',
 'MARIA VILMA VILLAGRA RODAS',
 'FREDDY ENRIQUE ZARACHO DUARTE',
 'MOISES ZACARIAS MARIN ACOSTA',
 'ALCIDES GUSTAVO CHILAVERT GONZALEZ',
 'SUSANA ELIZABETH MENDOZA BELLOSO',
 'RUBEN GARCIA AGUILAR',
 'BLAS ARIEL FRANCO LEZCANO',
 'KARINA CONCEPCION LOPEZ VAZQUEZ',
 'JULIA YANINA ORTIZ GONZALEZ',
 'EDGAR MIGUEL RODRIGUEZ VAZQUEZ']

In [93]:
cedula

['13.371.132',
 '25.240.438',
 '34.154.431',
 '42.442.763',
 '54.379.520',
 '6724.881',
 '74.728.086',
 '83.336.137',
 '93.661.207',
 '105.040.135',
 '115.140.311',
 '124.164.134']